### Webscraping in Week3 Peer Graded Assignment

**Retreiving the URL and creating a Beautiful soup object**

1. Let's download all the dependencies that we will need

In [2]:
!pip install geocoder
from bs4 import BeautifulSoup
import pandas as pd
import requests
import geocoder # import geocoder
import csv
import numpy as np
print('Libraries imported.')

Libraries imported.


2. Initialization of data frame

In [3]:
neigh_toronto = pd.DataFrame(columns=['PostalCode', 'Borough','Neighborhood'])

3. After retreiving the URL and creating a Beautiful soup object

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# request HTML by utilizing request library
data = requests.get(url).text
soup = BeautifulSoup(data,'html.parser')
table=soup.find('table')
for row in table.findAll('td'):
    if row.span.text=='Not assigned':
        pass
    else:
        PostalCode = row.p.text[:3]
        Borough = (row.span.text).split('(')[0]
        Neighborhood = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        neigh_toronto = neigh_toronto.append({'PostalCode' : PostalCode, 'Borough' : Borough, 'Neighborhood' : Neighborhood}, ignore_index=True)
df = neigh_toronto        
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.shape

(103, 3)

**Part2 - Get the coordinate of the neigh_toronto initialize the variable to None**

In [6]:
from pathlib import Path
my_csv = Path("C:/Users/LENOVO-PC/Downloads/Geospatial_Coordinates.csv")
my_csv

WindowsPath('C:/Users/LENOVO-PC/Downloads/Geospatial_Coordinates.csv')

In [8]:
addt_df = pd.read_csv("Geospatial_Coordinates.csv")

addt_df.rename(columns = {"Postal Code": "PostalCode"}, inplace=True)
final_df = pd.merge(df,addt_df, on = 'PostalCode', how = 'left')
final_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**Part3 - Clustering neighborhood based on their coordinates**

In [11]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#import folium # map rendering library
kclusters = 5
drop_col = ['PostalCode','Borough', 'Neighborhood']
final_df_clustering = final_df.drop(drop_col,1)

In [12]:
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(final_df_clustering)

In [13]:
final_df_clustering = final_df
final_df_clustering.insert(0, 'Cluster Labels', kmeans.labels_)
final_df_clustering

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,0,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...,...
98,1,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,4,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,1,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [ ]:
# create map
map_clusters = folium.Map(location = [43.6, -79.3], zoom_start = 11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i +x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the maps
markers_colors = []
for lat, lon, poi, cluster in zip(final_df_clustering['Latitude'], final_df_clustering['Longitude'],\
                                 final_df_clustering['Neighborhood'], final_df_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
map_clusters